In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r'./使用数据\一清数据.csv',encoding='gb2312')
df

,样地号,样地类别,纵坐标,横坐标,GPS纵坐标,GPS横坐标,县代码,地貌,海拔,坡向,...,活立蓄积,林木蓄积,散生蓄积,四旁蓄积,枯损蓄积,采伐蓄积,x,y,Unnamed: 76,Unnamed: 77
0,1,11,3680,19364,3679845,19364015,322,3,1620,1,...,0.000,0.000,0.0,0.000,0.0,0.000,3680000,19364000,NaN,NaN
1,2,11,3680,19372,3680005,19371995,322,3,1500,5,...,0.000,0.000,0.0,0.000,0.0,0.000,3680000,19372000,NaN,NaN
2,3,11,3680,19376,3679985,19376010,322,3,1100,4,...,0.764,0.000,0.0,0.764,0.0,0.225,3680000,19376000,NaN,NaN
3,4,11,3680,19380,3680005,19379995,322,3,1680,7,...,0.000,0.000,0.0,0.000,0.0,0.000,3680000,19380000,NaN,NaN
4,5,11,3680,19392,3680000,19391985,322,3,1370,5,...,0.846,0.846,0.0,0.000,0.0,0.230,3680000,19392000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,11956,11,3272,19712,3272000,19712000,1023,6,30,9,...,0.859,0.000,0.0,0.859,0.0,0.006,3272000,19712000,NaN,NaN
5816,11965,11,3264,19688,3264000,19688000,1023,6,20,9,...,0.000,0.000,0.0,0.000,0.0,0.000,3264000,19688000,NaN,NaN
5817,11966,11,3264,19700,3264000,19700000,1023,6,20,9,...,0.000,0.000,0.0,0.000,0.0,0.000,3264000,19700000,NaN,NaN
5818,11967,11,3264,19704,3264000,19704000,1023,6,30,9,...,0.000,0.000,0.0,0.000,0.0,0.000,3264000,19704000,NaN,NaN


In [3]:
df=df[df['地类'].isin(['111','131','132'])]

In [4]:
df['地类'].unique()

array([131, 111, 132], dtype=int64)

面积估计

In [6]:
groupby_df = df.groupby('地类')['样地号'].agg([np.size])
groupby_df['面积成数估计值'] = groupby_df['size'].apply(lambda x:x/np.sum(groupby_df['size']))
groupby_df['面积成数估计值标准差'] = groupby_df['面积成数估计值'].apply(lambda x:np.sqrt(x*(1-x)/(np.sum(groupby_df['size'])-7)))
groupby_df['面积成数估计值标准误'] = groupby_df.apply(lambda x:x['面积成数估计值标准差']/np.sqrt(x['size']),axis=1)
groupby_df['面积估计值'] = groupby_df['面积成数估计值'].apply(lambda x:x*667*np.sum(groupby_df['size']))
groupby_df['面积估计值误差限'] = groupby_df['面积成数估计值标准差'].apply(lambda x:x*2)
groupby_df['抽样精度'] = groupby_df.apply(lambda x:1-2*x['面积成数估计值标准误']/x['面积成数估计值'],axis=1)

In [7]:
groupby_df

,size,面积成数估计值,面积成数估计值标准差,面积成数估计值标准误,面积估计值,面积估计值误差限,抽样精度
地类,,,,,,,
111,1897,0.788773,0.008335,0.000191,1265299.0,0.016671,0.999515
131,349,0.145114,0.007193,0.000385,232783.0,0.014385,0.994694
132,159,0.066112,0.005074,0.000402,106053.0,0.010148,0.987827


In [8]:
groupby_df

,size,面积成数估计值,面积成数估计值标准差,面积成数估计值标准误,面积估计值,面积估计值误差限,抽样精度
地类,,,,,,,
111,1897,0.788773,0.008335,0.000191,1265299.0,0.016671,0.999515
131,349,0.145114,0.007193,0.000385,232783.0,0.014385,0.994694
132,159,0.066112,0.005074,0.000402,106053.0,0.010148,0.987827


In [9]:
groupby_df

,样地数目,面积成数估计值,面积成数估计值标准差,面积成数估计值标准误,面积估计值,面积估计值误差限,抽样精度
地类,,,,,,,
111,1897,0.788773,0.008335,0.000191,1265299.0,0.016671,0.999515
131,349,0.145114,0.007193,0.000385,232783.0,0.014385,0.994694
132,159,0.066112,0.005074,0.000402,106053.0,0.010148,0.987827


In [10]:
groupby_df.to_excel(r'./输出数据/面积估计.xlsx')

蓄积估计

In [11]:
grouped_df1 = df.groupby('地类')['活立蓄积'].agg([np.size,np.mean,np.var,np.std])
grouped_df1['面积成数估计值标准误'] = grouped_df1.apply(lambda x:x['std']/np.sqrt(x['size']),axis=1)
grouped_df1['总体总量估计值'] = grouped_df1.apply(lambda x:x['size']*x['mean'],axis=1)
grouped_df1['总体总量估计值误差限'] = grouped_df1.apply(lambda x:x['size']*x['面积成数估计值标准误']*2,axis=1)
grouped_df1['抽样精度'] = grouped_df1.apply(lambda x:1-(2*x['面积成数估计值标准误']/x['mean']),axis=1)
grouped_df1 = grouped_df1.rename(columns={'mean':'样本平均数','size':'样地数目','var':'样地方差','std':'样地标准差'})

In [12]:
grouped_df1

,样地数目,样本平均数,样地方差,样地标准差,面积成数估计值标准误,总体总量估计值,总体总量估计值误差限,抽样精度
地类,,,,,,,,
111,1897.0,3.978224,12.231025,3.497288,0.080297,7546.691,304.645731,0.959632
131,349.0,0.124023,0.083276,0.288576,0.015447,43.284,10.782077,0.750899
132,159.0,0.112239,0.066385,0.257653,0.020433,17.846,6.497770,0.635898


In [13]:
grouped_df1

,样地数目,样本平均数,样地方差,样地标准差,面积成数估计值标准误,总体总量估计值,总体总量估计值误差限,抽样精度
地类,,,,,,,,
111,1897.0,3.978224,12.231025,3.497288,0.080297,7546.691,304.645731,0.959632
131,349.0,0.124023,0.083276,0.288576,0.015447,43.284,10.782077,0.750899
132,159.0,0.112239,0.066385,0.257653,0.020433,17.846,6.497770,0.635898


In [14]:
groupby_df.to_excel(r'./输出数据/蓄积估计.xlsx')